In [ ]:
import torch
from diffusers import DiffusionPipeline
import matplotlib.pyplot as plt

In [ ]:
# Load RealVisXL pipeline from Hugging Face model hub
pipe = DiffusionPipeline.from_pretrained(
    "SG161222/RealVisXL_V3.0",  # model repo id
    torch_dtype=torch.float16,  # Use half-precision to reduce memory usage
    use_safetensors=True,       # load weights using the safer/faster safetensors format if available
    verbose=False,              # disable progress bars
)

# Move the pipeline to the GPU for inference
pipe = pipe.to("cuda")

# Enable attention slicing to reduce peak memory usage during generation
pipe.enable_attention_slicing()

In [ ]:
prompt = """" Light-medium skintone male, oval face, neutral expression.
Short dark brown thick hair, straight hairline, smooth forehead.
Short well-groomed beard. Medium round dark brown eyes, average lashes.
Medium-thick slightly arched dark brown brows. Straight medium nose, average lips with thinner upper lip.
Strong jawline, defined chin, average ears, no scars.

"""

image = pipe(prompt, verbose=False).images[0]
image.save("generated_face_with_formated_prompt.png")


In [ ]:


prompt = """ Light-medium skintone male, oval face, neutral expression.
Short dark brown thick hair, straight hairline, smooth forehead.
Short well-groomed beard. Medium round dark brown eyes, average lashes.
Medium-thick slightly arched dark brown brows. Straight medium nose, average lips with thinner upper lip.
Strong jawline, defined chin, average ears, no scars.

"""

# Generate a smaller image to reduce memory usage
image = pipe(prompt, height=768, width=768, verbose=False).images[0]
image.save("generated_face_with_formated_prompt.png")
plt.imshow(image)
plt.axis('off')

In [ ]:
import matplotlib.pyplot as plt

prompts = [
    "Light-medium skintone male, oval face, neutral expression. Short dark brown thick hair, straight hairline, smooth forehead. Short well-groomed beard. Medium round dark brown eyes, average lashes. Medium-thick slightly arched dark brown brows. Straight medium nose, average lips with thinner upper lip. Strong jawline, defined chin, average ears, no scars. Mugshot style, plain grey background, front-facing, studio lighting.",
    "Female, early 30s, neutral expression, medium length blonde hair pulled back, light makeup, plain blue background, front-facing, professional mugshot style.",
    "Male, late 40s, slightly wrinkled face, short grey hair, stubble beard, dark clothing, white background, straight on, mugshot photography.",
    "Female, mid-20s, dark skin tone, natural curly black hair, no visible jewelry, neutral expression, plain white wall background, clear focus, mugshot.",
    "Male, elderly, bald head with some grey hair on sides, glasses, slight frown, simple black t-shirt, beige background, front-view, mugshot camera style."
]

for i, p in enumerate(prompts):
    print(f"Generating image for prompt {i+1}:")
    # Generate a smaller image to reduce memory usage, similar to previous successful runs
    image = pipe(p, height=768, width=768, verbose=False).images[0]
    filename = f"generated_image_realvisxl_mugshot_{i+1}.png"
    image.save(filename)
    print(f"Image saved as {filename}")

    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.title(f"RealVisXL - Mugshot Prompt {i+1}")
    plt.axis('off')
    plt.show()

In [ ]:

#!/usr/bin/env python3
"""
Download any Hugging Face model and save it into a target directory.

Usage:
------
python download_model.py --repo SG161222/RealVisXL_V4.0 --out /path/to/save/RealVizXL
"""

import argparse
import os
import shutil
from pathlib import Path

def main():
    try:
        from huggingface_hub import snapshot_download
    except ImportError:
        print("ERROR: Install dependency first:\n  pip install huggingface_hub")
        return

    parser = argparse.ArgumentParser(description="Download and save a HuggingFace model repo.")
    parser.add_argument("--repo", type=str, required=True,
                        help="HuggingFace model repo ID (e.g., SG161222/RealVisXL_V4.0)")
    parser.add_argument("--out", type=str, required=True,
                        help="Target directory to save the full model")
    parser.add_argument("--revision", type=str, default=None,
                        help="Optional: branch, tag or commit hash")
    parser.add_argument("--token", type=str, default=None,
                        help="HF token (or set env var HUGGINGFACE_TOKEN)")
    parser.add_argument("--offline", action="store_true",
                        help="Use only local cache, no internet")
    args = parser.parse_args()

    repo = args.repo
    out_dir = Path(args.out).resolve()
    out_dir.parent.mkdir(parents=True, exist_ok=True)

    token = args.token or os.getenv("HUGGINGFACE_TOKEN")

    print(f"[INFO] Downloading model '{repo}'...")
    try:
        cached_path = snapshot_download(
            repo_id=repo,
            revision=args.revision,
            local_files_only=args.offline,
            token=token
        )
    except Exception as e:
        print(f"[ERROR] Download failed: {e}")
        return

    cached_path = Path(cached_path)

    # Delete old folder if exists
    if out_dir.exists():
        print(f"[INFO] Removing existing directory: {out_dir}")
        shutil.rmtree(out_dir)

    print(f"[INFO] Saving model to: {out_dir}")
    shutil.copytree(cached_path, out_dir)

    print(f"[SUCCESS] Model saved at: {out_dir}")

if __name__ == "__main__":
    main()
